In [6]:
%pylab inline
import pandas as pd

data = pd.read_csv('./train.csv')

Populating the interactive namespace from numpy and matplotlib


In [7]:
training = data.ix[:,'feat_1':'feat_93']
target = data.ix[:, 'target']

In [22]:
data = data.reindex(np.random.permutation(data.index))
inputs = array(data.ix[:,'feat_1':'feat_93'])
targets = array(data.ix[:, 'target'])
training_count = int(len(inputs)*0.8)
print inputs[:training_count].shape

(49502, 93)


In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 30)
new_training = pca.fit_transform(training)
print(new_training.shape)

(61878, 30)


In [5]:
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = DecisionTreeClassifier()
#ada = AdaBoostClassifier(clf, n_estimators=600)
cross_val_score(clf, training, target, cv=10)

array([ 0.70542636,  0.71103214,  0.72035541,  0.7092084 ,  0.71574014,
        0.70788878,  0.72567087,  0.72065955,  0.72276107,  0.71890668])

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score

rfc = RandomForestClassifier(n_estimators=100)
#ada = AdaBoostClassifier(rfc)
cross_val_score(rfc, training, target, cv=10)

array([ 0.80813953,  0.81036989,  0.81147011,  0.80436187,  0.80833872,
        0.80730682,  0.81231814,  0.81506628,  0.81167152,  0.81287401])

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.svm import SVC

svc = SVC()
cross_val_score(svc, training, target, cv=10)

In [ ]:
from sklearn import grid_search

parameters = {'max_depth':(5, 10, 20, 50), 'min_samples_leaf':(1, 5, 10), 'n_estimators':(10, 100, 200, 500)}
dd = grid_search.GridSearchCV(rfc, parameters, cv=10)
dd.fit(training, target)
print(dd.best_estimator_)

In [12]:
### plot decision tree
from sklearn.externals.six import StringIO
import pydot
from sklearn import tree

dot_data = StringIO()
tree.export_graphviz(model, out_file=dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue().replace('\n', '\t'))
graph.write_png('./tree.png')
from IPython.display import Image
Image(filename='tree.png')

True

In [47]:
test = pd.read_csv('./test.csv')
ids = test['id']
testdata = test.ix[:, 'feat_1':'feat_93']

rfc.fit(training, target)
result = rfc.predict_proba(testdata)
result = pd.DataFrame(result)
result = result.set_index(np.arange(result.shape[0])+1)
result.to_csv('./output_randomforest.csv')